In [1]:
''' imports '''

import os
import pickle
import random
import re
import shutil
from typing import Dict, List, Tuple

import numpy as np
import torch

from torch.nn.utils.rnn import pad_sequence

from torch.utils.data import DataLoader, Dataset, RandomSampler, SequentialSampler

from transformers import (
    WEIGHTS_NAME,
    AdamW,
    GPT2Config,
    GPT2LMHeadModel,
    GPT2Tokenizer,
    PreTrainedModel,
    PreTrainedTokenizer,
    get_linear_schedule_with_warmup,
)


In [2]:

class LineByLineTextDataset(Dataset):

    def __init__(self, tokenizer: PreTrainedTokenizer, file_path: str, block_size = 512):

        # no feature cache
        with open(file_path, encoding = 'utf-8') as f:

            lines = [line for line in f.read().splitlines()
                     if (len(line) > 0 and not line.isspace())]

        self.examples = tokenizer.batch_encode_plus(
            lines, add_special_tokens = True, max_length = block_size)['input_ids']

    def __len__(self):
        return len(self.examples)

    def __getitem__(self, i):
        return torch.tensor(self.examples[i], dtype=torch.long)


In [3]:
''' config parameters '''

# Optional input sequence length after tokenization.
# The training dataset will be truncated in block of this size for training
# Default to the model max input length for single sentence inputs (take into account special tokens)
block_size: int = -1

# Number of updates steps to accumulate before performing a backward/update pass
gradient_accumulation_steps: int = 1

# Max gradient norm
max_grad_norm: float = 1.

# If > 0: set total number of training steps to perform. Override num_train_epochs
max_steps: int = -1

# Linear warmup over warmup_steps
warmup_steps: int = 0


''' init train env '''

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = 1


# Set seed
seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
if n_gpu > 0:
    torch.cuda.manual_seed_all(seed)
    


In [4]:
''' Load pretrained model and tokenizer '''

# load pretrained model and tokenizer
config_class, model_class, tokenizer_class = GPT2Config, GPT2LMHeadModel, GPT2Tokenizer

# init pretrained tokeniser
tokenizer = tokenizer_class.from_pretrained('gpt2', cache_dir = None)


# Our input block size will be the max possible for the model
if block_size <= 0:
    block_size = tokenizer.max_len
else:
    block_size = min(block_size, tokenizer.max_len)


# init config
config = config_class()

# Training new model from scratch
#model = model_class(config = config)

model = model_class.from_pretrained('gpt2', config = config)

# push model to device
model.to(device)



GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): Laye

In [5]:
''' init dataset '''

# get training dataset
#file_path = '../data/lyrics/eurovision-lyrics-lines.txt'
#file_path = '../data/lyrics/wikitext-2/wiki.train.tokens'
#file_path = '../data/lyrics/eurovision-lyrics-en-lines'
file_path = '../data/lyrics/eurovision-lyrics-lines-full'
train_dataset = LineByLineTextDataset(
    tokenizer, file_path = file_path, block_size = block_size)
#train_dataset = TextDataset(tokenizer, args, file_path=file_path, block_size = block_size)


In [6]:
""" init dataloader, optimiser """

train_batch_size = 1

def collate(examples: List[torch.Tensor]):

    if tokenizer._pad_token is None:

        return pad_sequence(examples, batch_first = True)

    return pad_sequence(examples, batch_first = True, padding_value = tokenizer.pad_token_id)

# init random sampler on dataset
train_sampler = RandomSampler(train_dataset)

# init dataloader
train_dataloader = DataLoader(
    train_dataset, sampler = train_sampler, batch_size = train_batch_size, collate_fn = collate)


In [7]:
''' init optimiser, learning rate scheduler '''

# The initial learning rate for Adam
learning_rate: float = 5e-5

# Weight decay if we apply some
weight_decay: float = 0.

# Epsilon for Adam optimizer
adam_epsilon: float = 1e-8


# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ["bias", "LayerNorm.weight"]
optimizer_grouped_parameters = [
    {
        "params": [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
        "weight_decay": weight_decay,
    },
    {"params": [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], "weight_decay": 0.0},
]

# init optimiser on model parameters
optimizer = AdamW(optimizer_grouped_parameters, lr = learning_rate, eps = adam_epsilon)



# set training epochs
num_train_epochs = 3

# get total steps
if max_steps > 0:
    t_total = max_steps
    num_train_epochs = max_steps // (len(train_dataloader) // gradient_accumulation_steps) + 1
else:
    t_total = len(train_dataloader) // gradient_accumulation_steps * num_train_epochs
    

# init learning rate scheduler
scheduler = get_linear_schedule_with_warmup(
    optimizer, num_warmup_steps = warmup_steps, num_training_steps = t_total)


In [8]:
''' perform training '''

global_step = 0
epochs_trained = 0
steps_trained_in_current_epoch = 0

tr_loss, logging_loss = 0.0, 0.0

model.resize_token_embeddings(len(tokenizer))

# zero gradients
model.zero_grad()

# iterate epochs
for epoch in range(num_train_epochs):

    # get data batch from dataloader
    for step, batch in enumerate(train_dataloader):

        # Skip past any already trained steps if resuming training
        if steps_trained_in_current_epoch > 0:
            steps_trained_in_current_epoch -= 1
            continue


        # unpack batch data
        inputs, labels = (batch, batch)

        # push data to device
        inputs = inputs.to(device)
        labels = labels.to(device)


        # set model to train
        model.train()

        try:
        
            # perform forward pass through model
            outputs = model(inputs, labels=labels)

            # obtain loss; model outputs are always tuple in transformers
            loss = outputs[0]

            # gradient accumulation
            if gradient_accumulation_steps > 1:
                loss = loss / gradient_accumulation_steps

            # backprop loss
            loss.backward()

            # store loss
            tr_loss += loss.item()


            if (step + 1) % gradient_accumulation_steps == 0:

                # perform gradient normalisation
                torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)

                # step optimiser
                optimizer.step()

                # Update learning rate schedule
                scheduler.step()


                model.zero_grad()
                global_step += 1

        except:
            pass
            
        if global_step % 100 == 0:
            print(global_step, tr_loss / global_step)

        if max_steps > 0 and global_step > max_steps:
            break
    if max_steps > 0 and global_step > max_steps:
        break



100 5.659765508174896
200 5.494545063972473
300 5.478431216875712
400 5.362243276834488
500 5.302845049858093
600 5.308865185976028
700 5.2795916533470155
800 5.229922005981207
900 5.2065229509936435
1000 5.189333032727242
1100 5.180007066943428
1200 5.188128788669904
1300 5.169527831352674
1400 5.1498353989635195
1500 5.1336630777517955
1600 5.11875241689384
1700 5.121118812911651
1800 5.11686218712065
1900 5.08969904749017
2000 5.084395640671254
2100 5.087113815773101
2200 5.0885325930877165
2300 5.067455465715865
2400 5.043693789119522
2500 5.020331008315086
2600 5.008030550181866
2700 5.003634709009418
2800 4.9934838668150565
2900 4.981640682282119
3000 4.968563390831153
3100 4.9634098595380785
3200 4.961379599589854
3300 4.953298058997501
3400 4.941046900486245
3500 4.939618603008134
3600 4.933954377323389
3700 4.916382098278484
3800 4.9079005726230776
3900 4.903903115231257
4000 4.894045700840652
4100 4.886051983651591
4200 4.884227139091208
4300 4.8762999769074975
4400 4.8722571

KeyboardInterrupt: 

In [9]:
''' saving '''

model.save_pretrained('../data/lyrics/model-en-lyrics-03')
#tokenizer.save_pretrained(output_dir)


In [25]:
# Load a trained model and vocabulary that you have fine-tuned
model = model_class.from_pretrained('../data/lyrics/models')
#tokenizer = tokenizer_class.from_pretrained(args.output_dir)
model.to(device)

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): Laye

In [16]:
''' perform text generation '''

# input prompt to model
prompt: str = 'I said in Dutch'

# max length desired output
length: int = 20

# Token at which text generation is stopped
stop_token: str = None

# temperature of 1.0 has no effect, lower tend toward greedy sampling
temperature:float = 1.


# primarily useful for CTRL model; in that case, use 1.2
repetition_penalty: float = 1.


k: int = 0
p: float = 0.9


# The number of samples to generate
num_return_sequences: int = 32


# Initialize the model and tokenizer
#model_class, tokenizer_class = (GPT2LMHeadModel, GPT2Tokenizer)

#tokenizer = tokenizer_class.from_pretrained('gpt2')
#model = model_class.from_pretrained('../data/lyrics/model-en-lyrics-02')
#model.to(device)


MAX_LENGTH = int(10000)  # Hardcoded max length to avoid infinite loop

def adjust_length_to_model(length, max_sequence_length):
    if length < 0 and max_sequence_length > 0:
        length = max_sequence_length
    elif 0 < max_sequence_length < length:
        length = max_sequence_length  # No generation bigger than model size
    elif length < 0:
        length = MAX_LENGTH  # avoid infinite loop
    return length

length = adjust_length_to_model(
    length, max_sequence_length = model.config.max_position_embeddings)


# encode prompt text, push to device
prompt_text = prompt
encoded_prompt = tokenizer.encode(prompt_text, add_special_tokens = False, return_tensors = "pt")
encoded_prompt = encoded_prompt.to(device)


# generate output sequence
output_sequences = model.generate(

    input_ids = encoded_prompt,
    
    max_length = length + len(encoded_prompt[0]),
    temperature = temperature,
    top_k = k,
    top_p = p,
    repetition_penalty = repetition_penalty,
    do_sample = True,
    num_return_sequences = num_return_sequences,
)


# Remove the batch dimension when returning multiple sequences
if len(output_sequences.shape) > 2:
    output_sequences.squeeze_()

generated_sequences = []

for generated_sequence_idx, generated_sequence in enumerate(output_sequences):
    
    generated_sequence = generated_sequence.tolist()

    # Decode text
    text = tokenizer.decode(generated_sequence, clean_up_tokenization_spaces = True)

    # Remove all text after the stop token
    text = text[: text.find(stop_token) if stop_token else None]

    # Add the prompt at the beginning of the sequence.
    # Remove the excess text that was used for pre-processing
    total_sequence = (
        prompt_text + text[len(tokenizer.decode(
            encoded_prompt[0], clean_up_tokenization_spaces = True)) :] )

    generated_sequences.append(total_sequence)
    print(total_sequence)

#return generated_sequences


I said in Dutch aniori wCijela lo dok zatoa mwčič,!
I said in Dutch, Netherlands, Amsterdam, Amsterdam, Amsterdam - Amsterdam"Quand on an islandyé?!
I said in Dutch:If I turn around, I'll know we'll be together forever more and more (am!
I said in Dutch, to which we were glad, they said in the junglier days – 'cause I'd!
I said in Dutch mag) alles joonan juieu zo foude arrivedra ik!
I said in Dutch, the time of my life, I love life, life is beautiful, sé bale!
I said in Dutch"E kosmarm and wörte Warte adiel, oft lies Sch!
I said in Dutch hanging day: "Slugwir ein liebeft" ihn inGonna!
I said in Dutch, I said in: "Eynu seni" Milia, a little girl,!
I said in Dutch: "Oh ah ah... well" I then repeated in Italian: "Ohh?" I!
I said in Dutch taste, all you got was me. I'll catch you when you’m around and!
I said in Dutch Eastern Europe, goodbye to the true path I'd tell you. Me about time to make good!
I said in Dutch language: "¿Que voy a la líve"When I’m trying!
I said in Dutch: "Hey baby, 

In [ ]:
# 'I said in Dutch' words to stay together all the time, jeg er viæm kren inte din!